In [3]:
import pandas as pd
import itertools

In [4]:
kline_size = "1m"
symbols = ["XBTUSD", "ETHUSD", "XRPZ18", "BCHZ18"]
features = ["close", "trades", "volume", "vwap", "turnover"]

In [ ]:
def prepare_df(symbols, kline_size, features, reduce = False):
    main_df = pd.DataFrame()
    selected_columns = []
    if symbols == ["XBTUSD", "ETHUSD", "XRPZ18", "LTCZ18", "EOSZ18", "BCHZ18", "ADAZ18", "TRXZ18"]:
        file_path = f'Merged-{kline_size}-data.csv'
        main_df = pd.read_csv(file_path)
        main_df.set_index("timestamp", inplace=True)
    else:
        for symbol in symbols:
            file_path = f'{symbol}-{kline_size}-data.csv'
            df = pd.read_csv(file_path)
            df.rename(columns={"close": f"{symbol}_close",
                               "high": f"{symbol}_high",
                               "low": f"{symbol}_low",
                               "open": f"{symbol}_open",
                               "trades": f"{symbol}_trades",
                               "volume": f"{symbol}_volume",
                               "vwap": f"{symbol}_vwap",
                               "turnover": f"{symbol}_turnover"}, inplace=True)
            df.set_index("timestamp", inplace=True)
            if len(main_df)==0:
                main_df = df
            else:
                main_df = main_df.join(df)
    
    for symbol, feature in itertools.product(symbols, features):
        selected_columns.append(f"{symbol}_{feature}")
    
    main_df.drop([c for c in main_df if c not in selected_columns], axis = "columns", inplace = True)
    
    if reduce:
        main_df[main_df==np.inf]=np.nan
        main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
        main_df.dropna(inplace=True)
    
    return main_df

In [ ]:
my_df = prepare_df(symbols, kline_size, features, reduce = True)
my_df.describe()